### **Ejercicio:** Análisis y Limpieza de Datos de Ventas

**Contexto:**

Tienes un conjunto de datos que representa las ventas diarias de una tienda en línea. Este conjunto contiene información sobre los productos vendidos, la cantidad, el precio por unidad, y la fecha de venta. Sin embargo, algunos de los datos están incompletos o tienen valores nulos. El objetivo de este ejercicio es limpiar y analizar los datos, y calcular algunas medidas clave para entender mejor las ventas.



**Objetivos:**
- **Manipular arrays con numpy:** Para operaciones matemáticas básicas con los datos.
- **Manejar datos con pandas:** Para cargar el dataset, analizar su estructura y calcular medidas de tendencia central.
- **Trabajar con archivos JSON:** Para guardar y leer la información transformada.
- **Manejar datos nulos:** Para identificar y lidiar con valores faltantes.
- **Aplicar ciclos, condicionales y operadores lógicos/matemáticos:** Para realizar cálculos y aplicar reglas de negocio.

In [5]:
import pandas as pd

# Ruta del archivo CSV en Windows
file_path = r"C:\Users\Andy\Documents\mitic-data-science-team-1-septiembre-2024\clases\ds-fundamentals\data\ejercicios_integracion\ventas_tienda_simulada.csv"

# 1. Cargar el archivo CSV
df = pd.read_csv(file_path)

# 2. Verificar valores nulos en cada columna
print("Valores nulos en cada columna antes de la limpieza:")
print(df.isnull().sum())

# 3. Completar la columna 'total' cuando sea posible (cantidad * precio_unitario)
df['total'] = df.apply(
    lambda row: row['cantidad'] * row['precio_unitario'] 
    if pd.isnull(row['total']) and not pd.isnull(row['cantidad']) and not pd.isnull(row['precio_unitario']) 
    else row['total'],
    axis=1
)

# 4. Rellenar valores nulos en 'cantidad' y 'precio_unitario' con la mediana por producto
df['cantidad'] = df.groupby('producto')['cantidad'].transform(lambda x: x.fillna(x.median()))
df['precio_unitario'] = df.groupby('producto')['precio_unitario'].transform(lambda x: x.fillna(x.median()))

# 5. Recalcular 'total' en caso de que se hayan completado 'cantidad' o 'precio_unitario'
df['total'] = df.apply(
    lambda row: row['cantidad'] * row['precio_unitario'] 
    if pd.isnull(row['total']) 
    else row['total'],
    axis=1
)

# 6. Verificación de valores nulos
print("\nValores nulos después de la limpieza:")
print(df.isnull().sum())

# 7. Estadísticas descriptivas generales de las ventas totales
print("\nEstadísticas de ventas totales:")
print(df['total'].describe())

# 8. Resumen de ventas por producto (total, promedio, mediana, cantidad)
ventas_por_producto = df.groupby('producto')['total'].agg(
    total_ventas='sum', 
    promedio_ventas='mean', 
    mediana_ventas='median', 
    cantidad_ventas='count'
)
print("\nResumen de ventas por producto:")
print(ventas_por_producto)

# 9. Guardar el DataFrame limpio en un nuevo archivo CSV
output_path = r"C:\Users\Andy\Documents\mitic-data-science-team-1-septiembre-2024\clases\ds-fundamentals\data\ejercicios_integracion\ventas_tienda_limpias.csv"
df.to_csv(output_path, index=False)

print(f"\nArchivo limpio guardado en: {output_path}")

Valores nulos en cada columna antes de la limpieza:
fecha              0
producto           0
cantidad           4
precio_unitario    4
total              8
dtype: int64

Valores nulos después de la limpieza:
fecha              0
producto           0
cantidad           0
precio_unitario    0
total              0
dtype: int64

Estadísticas de ventas totales:
count     20.000000
mean      74.600000
std       37.733693
min       12.000000
25%       39.750000
50%       90.000000
75%      100.000000
max      132.000000
Name: total, dtype: float64

Resumen de ventas por producto:
            total_ventas  promedio_ventas  mediana_ventas  cantidad_ventas
producto                                                                  
Producto_A         233.0             46.6            21.0                5
Producto_B         414.0             82.8            90.0                5
Producto_C         461.0             92.2            99.0                5
Producto_D         384.0             76.8   

### Algunos Análisis Adicionales

In [6]:
# 1. Promedio y mediana de precio unitario
print("\nPromedio y mediana del precio unitario:")
print(df['precio_unitario'].describe()[['mean', '50%']])  # 50% es la mediana

# 2. Producto más vendido en términos de cantidad total
producto_mas_vendido = df.groupby('producto')['cantidad'].sum().idxmax()
cantidad_producto_mas_vendido = df.groupby('producto')['cantidad'].sum().max()
print(f"\nProducto más vendido: {producto_mas_vendido} con {cantidad_producto_mas_vendido} unidades vendidas.")

# 3. Correlación entre precio unitario y total de ventas
correlacion_precio_ventas = df['precio_unitario'].corr(df['total'])
print(f"\nCorrelación entre precio unitario y total de ventas: {correlacion_precio_ventas:.2f}")

# 4. Promedio de ventas totales por día (si la columna 'fecha' está disponible)
if 'fecha' in df.columns:
    df['fecha'] = pd.to_datetime(df['fecha'], errors='coerce')
    ventas_diarias = df.groupby(df['fecha'].dt.date)['total'].sum()
    promedio_ventas_diarias = ventas_diarias.mean()
    print(f"\nPromedio de ventas diarias: {promedio_ventas_diarias:.2f}")



Promedio y mediana del precio unitario:
mean    17.15
50%     18.00
Name: precio_unitario, dtype: float64

Producto más vendido: Producto_C con 24.5 unidades vendidas.

Correlación entre precio unitario y total de ventas: 0.67

Promedio de ventas diarias: 74.60
